In [1]:
import numpy as np
from numpy import load
import torch
import torch.nn as nn
import torch.nn.functional as F

from timeit import default_timer as timer


from tensorboardX import SummaryWriter
from datetime import datetime
from source.Data_load import Data_load
from source.Attention_LSTM import RNNModel
from source.train import train_epoch_lstm,evaluate_lstm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
mbrnlist1=[(5,194),(2,155),(12,100),(17,29),(42,1),(44,1),(50,92),(2,83),(4,10118),(8,298)]
mbrnlist2=[(5,194),(12,100),(2,155),(17,29),(42,1),(44,1),(2,83),(4,10118),(4,9997),(50,91)]

mbrnlist=mbrnlist1+mbrnlist2
mbrnlist=set(mbrnlist)
mbrnlist=list(mbrnlist)

# load array
MBR_NO,BRN_NO=mbrnlist[2]
featnorm=True

In [4]:
bptt = 39
TGT_VOCAB_SIZE = 3
EMB_SIZE =128
NHEAD = 16
FFN_HID_DIM = 128
BATCH_SIZE = 32
NUM_LAYERS = 6

In [ ]:
MBR_NO=42
BRN_NO=1
Val_loss=[]
Train_loss=[]
Accuracy=[]
F1score=[]

if featnorm==True:
    Data_train = load('OldData/Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_train.npy',allow_pickle=True)
    Data_test =  load('OldData/Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_test.npy',allow_pickle=True)
else:
    Data_train = load('OldData/Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'train.npy',allow_pickle=True)
    Data_test =  load('OldData/Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'test.npy',allow_pickle=True)

Xtrain_data,Ytrain_data,Xtest_data,Ytest_data = Data_load(Data_train,Data_test)
torch.manual_seed(0)

SRC_VOCAB_SIZE = Xtrain_data.shape[1]


lstm = RNNModel(rnn_type='LSTM',ntoken=SRC_VOCAB_SIZE,ninp=EMB_SIZE,nhid=FFN_HID_DIM,nlayers=NUM_LAYERS,proj_size=TGT_VOCAB_SIZE,
                attention_width=38)
summary = SummaryWriter()
for p in lstm.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

lstm = lstm.to(device)

loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(lstm.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

NUM_EPOCHS = 1000
best_val_loss=100000000
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss,_ = train_epoch_lstm(lstm, optimizer,Xtrain_data,Ytrain_data,loss_fn,device,BATCH_SIZE,bptt)
    end_time = timer()
    val_loss,acc,prec,reca,f1sc,confusion = evaluate_lstm(lstm,Xtest_data,Ytest_data,loss_fn,device,BATCH_SIZE,bptt)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_confusion=confusion
        best_acc=acc
        best_prec=prec
        best_reca=reca
        best_f1sc=f1sc
        best_model = lstm
    Val_loss.append(val_loss)
    Train_loss.append(train_loss)
    Accuracy.append(acc)
    F1score.append(f1sc)
now = datetime.now()
now.strftime("%m/%d/%Y, %H:%M:%S")

date_time = now.strftime("%m_%d_%Y")

PATH='best_model_alstm_seq_'+date_time+'_'+str(MBR_NO)+'_'+str(BRN_NO)
#     if featnorm==True:
#         torch.save(best_model.state_dict(), PATH+'norm')
#     else:
#         torch.save(best_model.state_dict(), PATH)
if featnorm==True:
    file_name='results/result_ALSTM_'+date_time+'_norm.txt'
else:
    file_name='results/result_ALSTM_'+date_time+'.txt'
text_to_append=PATH+'\t'+"Acc:"+str(best_acc)+'\t'+"prec:"+str(best_prec)+'\t'+"recall:"+str(best_reca)+'\t'+"f1sc:"+str(best_f1sc)
print(text_to_append)
with open(file_name, "a+") as file_object:
    # Move read cursor to the start of file.
    file_object.seek(0)
    # If file is not empty then append '\n'
    data = file_object.read(100)
    if len(data) > 0:
        file_object.write("\n")
    # Append text at the end of file
    file_object.write(text_to_append)

/Data/Min/TransLOB/source/Attention_LSTM.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = self.softmax(a)


torch.Size([468, 7])
torch.Size([468, 1])
74880
42754
Acc: 0.5709668803418804
Prec 0.28209975835212664
Recall 0.3425891674736208
F1 0.30941583038506726
torch.Size([312, 7])
torch.Size([312, 1])
7488
4961
Acc: 0.6625267094017094
Prec 0.22084223646723647
Recall 0.3333333333333333
F1 0.26567060272578785
Epoch: 1, Train loss: 0.001, Val loss: 0.001, Epoch time = 2.928s
torch.Size([468, 7])
torch.Size([468, 1])
74880
45111
Acc: 0.6024439102564103
Prec 0.2029284750337382
Recall 0.3333333333333333
F1 0.2522753772722316
torch.Size([312, 7])
torch.Size([312, 1])
7488
4961
Acc: 0.6625267094017094
Prec 0.22084223646723647
Recall 0.3333333333333333
F1 0.26567060272578785
Epoch: 2, Train loss: 0.001, Val loss: 0.001, Epoch time = 2.907s
torch.Size([468, 7])
torch.Size([468, 1])
74880
45111
Acc: 0.6024439102564103
Prec 0.2029284750337382
Recall 0.3333333333333333
F1 0.2522753772722316
torch.Size([312, 7])
torch.Size([312, 1])
7488
4961
Acc: 0.6625267094017094
Prec 0.22084223646723647
Recall 0.333333

torch.Size([468, 7])
torch.Size([468, 1])
74880
49679
Acc: 0.6634481837606837
Prec 0.5356274186723634
Recall 0.47109841209666925
F1 0.5012948286411449
torch.Size([312, 7])
torch.Size([312, 1])
7488
5373
Acc: 0.7175480769230769
Prec 0.5451866922318621
Recall 0.5148123023317807
F1 0.5295643065096914
Epoch: 24, Train loss: 0.001, Val loss: 0.001, Epoch time = 2.838s
torch.Size([468, 7])
torch.Size([468, 1])
74880
49797
Acc: 0.6650240384615385
Prec 0.5345769116453859
Recall 0.47972503030623165
F1 0.5056678184932968
torch.Size([312, 7])
torch.Size([312, 1])
7488
5419
Acc: 0.7236912393162394
Prec 0.5442263263466419
Recall 0.5108995645186121
F1 0.5270366229039327
Epoch: 25, Train loss: 0.001, Val loss: 0.001, Epoch time = 2.858s
torch.Size([468, 7])
torch.Size([468, 1])
74880
50038
Acc: 0.6682425213675214
Prec 0.5394999081701822
Recall 0.4787554196533466
F1 0.5073157937529021
torch.Size([312, 7])
torch.Size([312, 1])
7488
5355
Acc: 0.7151442307692307
Prec 0.5334096461508963
Recall 0.514502141

torch.Size([468, 7])
torch.Size([468, 1])
74880
51296
Acc: 0.685042735042735
Prec 0.5627926700003639
Recall 0.5089600177881652
F1 0.5345243741361204
torch.Size([312, 7])
torch.Size([312, 1])
7488
5705
Acc: 0.7618856837606838
Prec 0.6087756999655968
Recall 0.5628692060332045
F1 0.5849231164451362
Epoch: 47, Train loss: 0.001, Val loss: 0.000, Epoch time = 2.857s
torch.Size([468, 7])
torch.Size([468, 1])
74880
51764
Acc: 0.691292735042735
Prec 0.5667680406774028
Recall 0.5223896239424152
F1 0.5436747190047927
torch.Size([312, 7])
torch.Size([312, 1])
7488
5675
Acc: 0.7578792735042735
Prec 0.5947127219651649
Recall 0.5605450401252215
F1 0.5771236126450904
Epoch: 48, Train loss: 0.001, Val loss: 0.001, Epoch time = 2.934s
torch.Size([468, 7])
torch.Size([468, 1])
74880
51552
Acc: 0.6884615384615385
Prec 0.5631904377156193
Recall 0.5177719281771841
F1 0.5395270142011331
torch.Size([312, 7])
torch.Size([312, 1])
7488
5702
Acc: 0.7614850427350427
Prec 0.5986529132520485
Recall 0.5561271643833

torch.Size([468, 7])
torch.Size([468, 1])
74880
50183
Acc: 0.670178952991453
Prec 0.5527819004442084
Recall 0.4957406101537303
F1 0.5227096868943257
torch.Size([312, 7])
torch.Size([312, 1])
7488
5419
Acc: 0.7236912393162394
Prec 0.5739550778239649
Recall 0.5330429019142214
F1 0.552742978490434
Epoch: 70, Train loss: 0.001, Val loss: 0.001, Epoch time = 2.873s
torch.Size([468, 7])
torch.Size([468, 1])
74880
50145
Acc: 0.6696714743589743
Prec 0.555388457427184
Recall 0.4969154726264926
F1 0.5245273916246809
torch.Size([312, 7])
torch.Size([312, 1])
7488
5535
Acc: 0.7391826923076923
Prec 0.6112650022019163
Recall 0.5264856039042679
F1 0.5657166379040091
Epoch: 71, Train loss: 0.001, Val loss: 0.001, Epoch time = 2.856s
torch.Size([468, 7])
torch.Size([468, 1])
74880
50032
Acc: 0.6681623931623931
Prec 0.5562739291295621
Recall 0.49285978824584803
F1 0.5226503473806051
torch.Size([312, 7])
torch.Size([312, 1])
7488
5557
Acc: 0.7421207264957265
Prec 0.6154120174212678
Recall 0.5353868820853

...

torch.Size([468, 7])
torch.Size([468, 1])
74880
60378
Acc: 0.8063301282051282
Prec 0.7436271353664328
Recall 0.6968619694130673
F1 0.7194854419114446
torch.Size([312, 7])
torch.Size([312, 1])
7488
6246
Acc: 0.8341346153846154
Prec 0.7492799877021078
Recall 0.6869958290254446
F1 0.7167874308381926
Epoch: 714, Train loss: 0.000, Val loss: 0.000, Epoch time = 2.693s
torch.Size([468, 7])
torch.Size([468, 1])
74880
60326
Acc: 0.8056356837606837
Prec 0.7435521636325647
Recall 0.6947302752359183
F1 0.7183126002693959
torch.Size([312, 7])
torch.Size([312, 1])
7488
6271
Acc: 0.8374732905982906
Prec 0.7561750491637039
Recall 0.694788981360512
F1 0.7241834822726386
Epoch: 715, Train loss: 0.000, Val loss: 0.000, Epoch time = 2.823s
torch.Size([468, 7])
torch.Size([468, 1])
74880
60435
Acc: 0.8070913461538461
Prec 0.7444319790324614
Recall 0.6973588063570709
F1 0.7201269443150907
torch.Size([312, 7])
torch.Size([312, 1])
7488
6258
Acc: 0.8357371794871795
Prec 0.7520285457231023
Recall 0.6927338421

torch.Size([468, 7])
torch.Size([468, 1])
74880
60555
Acc: 0.8086939102564102
Prec 0.7461620696186869
Recall 0.6997048818824495
F1 0.7221871172111054
torch.Size([312, 7])
torch.Size([312, 1])
7488
6267
Acc: 0.8369391025641025
Prec 0.7534001577582327
Recall 0.6933122499488918
F1 0.7221083550600946
Epoch: 737, Train loss: 0.000, Val loss: 0.000, Epoch time = 2.711s
torch.Size([468, 7])
torch.Size([468, 1])
74880
60637
Acc: 0.8097889957264958
Prec 0.7472464842270362
Recall 0.7016016368656607
F1 0.7237050576155679
torch.Size([312, 7])
torch.Size([312, 1])
7488
6273
Acc: 0.8377403846153846
Prec 0.756837525056278
Recall 0.6927369423604274
F1 0.7233699609864931
Epoch: 738, Train loss: 0.000, Val loss: 0.000, Epoch time = 2.788s
torch.Size([468, 7])
torch.Size([468, 1])
74880
60464
Acc: 0.8074786324786325
Prec 0.7406935655373172
Recall 0.7011685822153998
F1 0.7203893354344996
torch.Size([312, 7])
torch.Size([312, 1])
7488
6252
Acc: 0.8349358974358975
Prec 0.752793982410778
Recall 0.68602538216

torch.Size([468, 7])
torch.Size([468, 1])
74880
60623
Acc: 0.8096020299145299
Prec 0.745956524116516
Recall 0.7017343231177645
F1 0.7231700021122968
torch.Size([312, 7])
torch.Size([312, 1])
7488
6283
Acc: 0.8390758547008547
Prec 0.7555086429065477
Recall 0.6967506208126943
F1 0.7249409649161098
Epoch: 760, Train loss: 0.000, Val loss: 0.000, Epoch time = 2.860s
torch.Size([468, 7])
torch.Size([468, 1])
74880
60595
Acc: 0.8092280982905983
Prec 0.7459862095529787
Recall 0.7009232485915163
F1 0.7227530022160217
torch.Size([312, 7])
torch.Size([312, 1])
7488
6258
Acc: 0.8357371794871795
Prec 0.7476357843080876
Recall 0.6936897437019751
F1 0.719653215766094
Epoch: 761, Train loss: 0.000, Val loss: 0.000, Epoch time = 2.940s
torch.Size([468, 7])
torch.Size([468, 1])
74880
60575
Acc: 0.8089610042735043
Prec 0.7458679948363259
Recall 0.7001877849575435
F1 0.7223063819151102
torch.Size([312, 7])
torch.Size([312, 1])
7488
6258
Acc: 0.8357371794871795
Prec 0.7510041062051345
Recall 0.69113637213

torch.Size([468, 7])
torch.Size([468, 1])
74880
60591
Acc: 0.8091746794871795
Prec 0.745271352332633
Recall 0.7013572336863992
F1 0.7226477605508349
torch.Size([312, 7])
torch.Size([312, 1])
7488
6256
Acc: 0.8354700854700855
Prec 0.7515654131753052
Recall 0.6921761331600939
F1 0.7206492641691957
Epoch: 783, Train loss: 0.000, Val loss: 0.000, Epoch time = 2.876s
torch.Size([468, 7])
torch.Size([468, 1])
74880
60721
Acc: 0.8109107905982906
Prec 0.7484973016790114
Recall 0.7045571620726854
F1 0.7258628604029311
torch.Size([312, 7])
torch.Size([312, 1])
7488
6264
Acc: 0.8365384615384616
Prec 0.7527506790374692
Recall 0.6933289186780431
F1 0.7218189305148962
Epoch: 784, Train loss: 0.000, Val loss: 0.000, Epoch time = 2.737s
torch.Size([468, 7])
torch.Size([468, 1])
74880
60778
Acc: 0.8116720085470085
Prec 0.7481264222358802
Recall 0.7051200824122473
F1 0.7259869028612885
torch.Size([312, 7])
torch.Size([312, 1])
7488
6266
Acc: 0.8368055555555556
Prec 0.7520381938105051
Recall 0.6934294743

torch.Size([468, 7])
torch.Size([468, 1])
74880
60595
Acc: 0.8092280982905983
Prec 0.7444023334445156
Recall 0.7054803583907191
F1 0.7244189173961173
torch.Size([312, 7])
torch.Size([312, 1])
7488
6262
Acc: 0.8362713675213675
Prec 0.7550750081921206
Recall 0.6904041324630438
F1 0.7212928797286192
Epoch: 806, Train loss: 0.000, Val loss: 0.000, Epoch time = 2.788s
torch.Size([468, 7])
torch.Size([468, 1])
74880
60656
Acc: 0.810042735042735
Prec 0.7494912587053291
Recall 0.7003193748507953
F1 0.7240714581672059
torch.Size([312, 7])
torch.Size([312, 1])
7488
6283
Acc: 0.8390758547008547
Prec 0.7532651970325791
Recall 0.6995353878761762
F1 0.7254067313208087
Epoch: 807, Train loss: 0.000, Val loss: 0.000, Epoch time = 2.730s
torch.Size([468, 7])
torch.Size([468, 1])
74880
60680
Acc: 0.8103632478632479
Prec 0.7479922336641448
Recall 0.7033629033020024
F1 0.7249913900702776
torch.Size([312, 7])
torch.Size([312, 1])
7488
6271
Acc: 0.8374732905982906
Prec 0.7546447737015445
Recall 0.6935847713

In [ ]:
import matplotlib.pyplot as plt
plt.plot(Val_loss);
plt.plot(Train_loss);

In [ ]:
plt.plot(F1score)
plt.plot(Accuracy)

In [ ]:
import pickle
with open("Val_loss_ALSTM", "wb") as fp:   #Pickling
    pickle.dump(Val_loss, fp)
with open("Train_loss_ALSTM", "wb") as fp:   #Pickling
    pickle.dump(Train_loss, fp)
with open("Accuracy_ALSTM", "wb") as fp:   #Pickling
    pickle.dump(Accuracy, fp)
with open("F1_ALSTM", "wb") as fp:   #Pickling
    pickle.dump(F1score, fp)